In [1]:
import pandas as pd
import sqlite3
DB_PATH = "alchemy.db"
EFFECT_SQL_QUERY = """
SELECT i.code, e.description, et."type",et.value FROM ingredients AS i
join properties props on i.id = props.ingredient_id 
join effects e on e.id =props.effect_id
left join effects_types et on e.id =et.effect_id
where i.code =? and props.ingredient_order =?
"""
SELECT_BY_EFFECT_TYPE_CODE_NOT_IN = """
SELECT i.code, props.ingredient_order, e.description, et."type",et.value FROM ingredients AS i
join properties props on i.id = props.ingredient_id 
join effects e on e.id =props.effect_id
left join effects_types et on e.id =et.effect_id
where i.code not in {ingredients} and et."type" in {effects_types}
"""
SELECT_ALL_EFFECTS="""
SELECT i.code, props.ingredient_order , e.description, et."type",et.value FROM ingredients AS i
join properties props on i.id = props.ingredient_id 
join effects e on e.id =props.effect_id
left join effects_types et on e.id =et.effect_id
"""
## where et."type" is not Null
def split_formula(formula):
    return [x[:-1] for x in formula], [int(x[-1]) for x in formula]
def get_by_code_order(ingredient_code,ingredient_order,cursor):
    cursor.execute(EFFECT_SQL_QUERY,(ingredient_code,ingredient_order))
    found_effects = cursor.fetchall()
    found_effects_df =pd.DataFrame(data=found_effects,columns=["code","description","effect_type","effect_value"])
    return found_effects_df
def evaluate_effects(all_effects_df):
    other_effects_df = all_effects_df[all_effects_df["effect_type"].isna()]
    grouped_effects = list(all_effects_df.groupby("effect_type"))
    score = 0
    other_effects_len = other_effects_df.shape[0]
    if other_effects_len > 4:
        score -= 1000
    else:
        pass
        # score -= 100 * other_effects_len
        
    for effect_type,effect_df in grouped_effects:
        effect_value = effect_df["effect_value"].sum()
        if effect_value>0:
            score+=10
        else:
            score-=100
    return score
def get_all_effects(ingredients,ingredients_effects_nums,cursor):
    all_effects_df=[]
    for ingredient_code,ingredient_effect_i in zip(ingredients,ingredients_effects_nums):
        all_effects_df.append(get_by_code_order(ingredient_code=ingredient_code,ingredient_order=ingredient_effect_i,cursor=cursor))
    for ingredient_code in ingredients:
        all_effects_df.append(get_by_code_order(ingredient_code=ingredient_code,ingredient_order=0,cursor=cursor))
    all_effects_df = pd.concat(all_effects_df)
    return all_effects_df
def get_by_ingredients_with_codes(ingredients_with_codes,cursor):
    ingredients = [x[:-1] for x in ingredients_with_codes]
    ingredients_effects_nums = [int(x[-1]) for x in ingredients_with_codes]
    return get_all_effects(ingredients,ingredients_effects_nums,cursor)
def calculate_score_by_formula(formula,cursor):
    all_effects_df = get_by_ingredients_with_codes(formula,cursor)
    return evaluate_effects(all_effects_df)

def potential_candidates_codes_generator_by_effect_type(potential_candidates,effect_type,positive_effects=True):
    potential_candidates=potential_candidates[potential_candidates["effect_type"]==effect_type]
    if positive_effects:
        potential_candidates=potential_candidates[potential_candidates["effect_value"]>=0]
    else:
        potential_candidates=potential_candidates[potential_candidates["effect_value"]<0]
    potential_candidates_codes =potential_candidates["code"].unique()
    potential_candidates_codes=[]
    for _,row in potential_candidates.iterrows():
        if row["ingredient_order"]==0:
            potential_candidates_codes.append(row["code"]+"1")
            potential_candidates_codes.append(row["code"]+"2")
            potential_candidates_codes.append(row["code"]+"3")
        else:
            potential_candidates_codes.append(row["code"]+str(row["ingredient_order"]))
    return potential_candidates_codes

def potential_candidates_codes_generator(all_ingredients_effects,formula,positive_effects=True):
    ingredients, ingredients_effects_nums = split_formula(formula)
    current_ingredients_types= []
    for ingredient, ingredient_effect_num in zip(ingredients, ingredients_effects_nums):
        _cur_ingredient_effects = all_ingredients_effects[all_ingredients_effects["code"]==ingredient]
        main_effect_type = _cur_ingredient_effects[_cur_ingredient_effects["ingredient_order"]==0]["effect_type"].values[0]
        if main_effect_type is not None:
            current_ingredients_types.append(main_effect_type)
        minor_effect_type = _cur_ingredient_effects[_cur_ingredient_effects["ingredient_order"]==ingredient_effect_num]["effect_type"].values[0]
        if minor_effect_type is not None:
            current_ingredients_types.append(minor_effect_type)
    current_effects_types = list(set(current_ingredients_types))

    potential_candidates=all_ingredients_effects[~all_ingredients_effects["code"].isin(ingredients)]
    all_candidates = []
    for effect_type in current_effects_types:
        all_candidates.extend(potential_candidates_codes_generator_by_effect_type(potential_candidates,effect_type,positive_effects=positive_effects))
    return all_candidates

    
def potential_candidates_with_max_score_one_step(all_ingredients_effects,formula,only_max_score=True):
    potential_candidates_codes = potential_candidates_codes_generator(all_ingredients_effects,formula)
    potential_candidates_scores=[]
    for potential_candidate_code in potential_candidates_codes:
        potential_candidates_scores.append(calculate_score_by_formula(formula+[potential_candidate_code],cursor))
    potential_candidates_scores_df = pd.DataFrame(data={"code":potential_candidates_codes,"score":potential_candidates_scores})
    potential_candidates_scores_df = potential_candidates_scores_df.sort_values(by="score",ascending=False)
    max_score = potential_candidates_scores_df["score"].max()
    if only_max_score:
        potential_candidates_scores_df=potential_candidates_scores_df[potential_candidates_scores_df["score"]==max_score]
    return potential_candidates_scores_df
    
def potential_candidates_with_max_score_several_steps(formula,steps=1,only_max_score=True, all_ingredients_effects=None):
    if all_ingredients_effects is None:
        cursor.execute(SELECT_ALL_EFFECTS)
        all_ingredients_effects = cursor.fetchall()
        all_ingredients_effects = pd.DataFrame(data=all_ingredients_effects, columns=["code","ingredient_order","description","effect_type","effect_value"])

    result = set()
    step_candidates = potential_candidates_with_max_score_one_step(all_ingredients_effects,formula,only_max_score=only_max_score)["code"].values.tolist()
    formulas = [formula+[x] for x in step_candidates]
    formulas = set([frozenset(x) for x in formulas])
    if steps >1:
        for current_formula in formulas:
            cur_res=potential_candidates_with_max_score_several_steps(list(current_formula),steps-1,only_max_score=only_max_score, all_ingredients_effects=all_ingredients_effects)
            cur_res = set([frozenset(x) for x in cur_res])
            result = result.union(cur_res)
    else:
        result = formulas
    result = [list(x) for x in result]
    return result
    
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()


In [3]:

formula = [
    # "MER1",
    "YN2","TD1"]

# ingredients, ingredients_effects_nums = split_formula(formula)


In [3]:
all_effects_df = get_all_effects(ingredients,ingredients_effects_nums,cursor)

In [4]:
current_effect_groups = [x for x in all_effects_df['effect_type'].unique() if x is not None]
if len(current_effect_groups)==1:
    current_effect_groups = (current_effect_groups[0],current_effect_groups[0])

In [5]:
current_effect_groups

('яд', 'яд')

In [6]:
query = SELECT_BY_EFFECT_TYPE_CODE_NOT_IN.format(ingredients=tuple(ingredients),effects_types=current_effect_groups)
cursor.execute(query)
all_ingredients_effects = cursor.fetchall()
all_ingredients_effects = pd.DataFrame(data=all_ingredients_effects, columns=["code","ingredient_order","description","effect_type","effect_value"])

In [7]:
cursor.execute(SELECT_ALL_EFFECTS)
all_ingredients_effects = cursor.fetchall()
all_ingredients_effects = pd.DataFrame(data=all_ingredients_effects, columns=["code","ingredient_order","description","effect_type","effect_value"])

In [8]:
potential_candidates_with_max_score_several_steps(formula,steps=2,only_max_score=True)

[['TD1', 'HV1', 'VZ3', 'YN2'],
 ['RK1', 'TD1', 'YN2', 'HV1'],
 ['RK1', 'TD1', 'VZ3', 'YN2']]

In [9]:
get_by_ingredients_with_codes

,code,description,effect_type,effect_value
0,YN,разрушает энергетические потоки (при силе 20 р...,None,None
0,TD,разжигает манию величия,None,None
0,YN,смертельный яд,яд,-4
0,TD,клептомания и тяга ко всему блестящему,None,None
